In [ ]:
#| default_exp utils

In [ ]:
#|export
import numpy as np
import matplotlib.pyplot as plt
def collect_baselines(env, actions, repetitions=10, cycles=5):
    results = {action: {'max_phenotype': [], 'gv': []} for action in actions}
    final_gen_averages = {}
    env.reset()
    for action in actions:
        final_gen_phenotypes = []
        for _ in range(repetitions):
            env.reset()
            cycle_max_phenotype = []
            cycle_gv = []
            cycle_max_phenotype.append(env.population.breeding_values.max())
            cycle_gv.append(env.population.breeding_values.var())
            for _ in range(cycles):
                env.step(np.array([action]))
                max_phenotype = env.population.breeding_values.max()
                gv = env.population.breeding_values.var()
                cycle_max_phenotype.append(max_phenotype)
                cycle_gv.append(gv)
            
            results[action]['max_phenotype'].append(cycle_max_phenotype)
            results[action]['gv'].append(cycle_gv)
            final_gen_phenotypes.append(cycle_max_phenotype[-1])
        
        final_gen_averages[action] = np.mean(final_gen_phenotypes)
    
    best_action = max(final_gen_averages, key=final_gen_averages.get)
    best_average = final_gen_averages[best_action]
    
    return results, best_action, best_average
def plot_best_run(results, best_action):
    plt.figure(figsize=(12, 10))
    
    max_phenotypes = np.array(results[best_action]['max_phenotype'])
    genetic_variances = np.array(results[best_action]['gv'])
    
    mean_phenotypes = np.mean(max_phenotypes, axis=0)
    std_phenotypes = np.std(max_phenotypes, axis=0)
    mean_gv = np.mean(genetic_variances, axis=0)
    std_gv = np.std(genetic_variances, axis=0)
    
    cycles = range(len(mean_phenotypes))
#     best_action_translated = scale_values(best_action)
    
    # Plot Max Phenotype
    plt.subplot(2, 1, 1)
    plt.plot(cycles, mean_phenotypes, label=f'Action {best_action:.3f}')
    plt.fill_between(cycles, mean_phenotypes - std_phenotypes, mean_phenotypes + std_phenotypes, alpha=0.3)
    plt.xlabel('Cycle')
    plt.ylabel('Max Phenotype')
    plt.title(f'Max Phenotype over Cycles for Best Action')
    plt.legend()
    plt.grid(True)
    
    # Plot Genetic Variance
    plt.subplot(2, 1, 2)
    plt.plot(cycles, mean_gv, label=f'Action {best_action:.3f}')
    plt.fill_between(cycles, mean_gv - std_gv, mean_gv + std_gv, alpha=0.3)
    plt.xlabel('Cycle')
    plt.ylabel('Genetic Variance')
    plt.title(f'Genetic Variance over Cycles for Best Action')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()